In [1]:
import gradio as gr
import os
import subprocess
import glob
import shutil
from IPython.display import display, HTML

UPLOAD_FOLDER = 'uploads'
OUTPUT_FOLDER = 'outputs'
SAVE_FOLDER = 'saved_results'

def process_images(images, text_input):
    # Clear previous uploads and outputs
    for folder in [UPLOAD_FOLDER, OUTPUT_FOLDER]:
        if os.path.exists(folder):
            shutil.rmtree(folder)
        os.makedirs(folder)
    
    # Save uploaded images
    image_paths = []
    for i, image in enumerate(images):
        file_extension = os.path.splitext(image.name)[1]
        image_path = os.path.join(UPLOAD_FOLDER, f"image_{i}{file_extension}")
        shutil.copy(image.name, image_path)
        image_paths.append(image_path)
    
    # Run the auto-labeling script
    cmd = [
        'python', 's2f2_labelanything.py',
        '--pipeline', 'open_vocabulary_detection_segmentation',
        '--image_dir', UPLOAD_FOLDER,
        '--output_dir', OUTPUT_FOLDER,
        '--text_input', text_input
    ]
    subprocess.run(cmd)
    
    # Collect output files
    output_files = glob.glob(os.path.join(OUTPUT_FOLDER, '*'))
    
    # Prepare results
    image_results = []
    box_coordinates = ""
    for file in output_files:
        if file.endswith(('.png', '.jpg', '.jpeg')):
            image_results.append((file, os.path.basename(file)))
        elif file.endswith('_box.txt'):
            with open(file, 'r') as f:
                content = f.read()
            box_coordinates += f"Bounding boxes for {os.path.basename(file)}:\n{content}\n\n"
    
    return image_results, box_coordinates

def save_results(save_button, use_box_coordinates):
    if not save_button:
        return "Click the Save button to save results."
    
    if not os.path.exists(SAVE_FOLDER):
        os.makedirs(SAVE_FOLDER)
    
    # Copy original images
    for image in glob.glob(os.path.join(UPLOAD_FOLDER, '*')):
        shutil.copy(image, SAVE_FOLDER)
    
    # Copy label files based on user selection
    label_type = "box" if use_box_coordinates else "mask"
    for label_file in glob.glob(os.path.join(OUTPUT_FOLDER, f'*_{label_type}.txt')):
        new_name = os.path.basename(label_file).replace(f'_{label_type}', '')
        shutil.copy(label_file, os.path.join(SAVE_FOLDER, new_name))
    
    return f"Results saved in {SAVE_FOLDER} folder with {'box' if use_box_coordinates else 'mask'} coordinates."

with gr.Blocks() as iface:
    gr.Markdown("# Auto-Label Image Processor")
    gr.Markdown("Upload images and provide text input for open vocabulary detection and segmentation.")
    
    with gr.Row():
        input_images = gr.File(file_count="multiple", label="Upload Images")
        text_input = gr.Textbox(label="Text Input (e.g., 'green basket')")
    
    process_button = gr.Button("Process Images")
    
    with gr.Row():
        output_gallery = gr.Gallery(label="Processed Images", show_label=True)
        output_text = gr.Textbox(label="Bounding Box Coordinates", show_label=True)
    
    with gr.Row():
        use_box_coordinates = gr.Checkbox(label="Use bounding box coordinates (uncheck for mask coordinates)", value=True)
        save_button = gr.Button("Save Results")
    
    save_output = gr.Textbox(label="Save Status")
    
    process_button.click(
        process_images,
        inputs=[input_images, text_input],
        outputs=[output_gallery, output_text]
    )
    
    save_button.click(
        save_results,
        inputs=[save_button, use_box_coordinates],
        outputs=[save_output]
    )

# Launch the interface
iface.launch(share=True)

# Display the public URL
public_url = iface.share_url
display(HTML(f'<p>Public URL: <a href="{public_url}" target="_blank">{public_url}</a></p>'))

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://a877bb8acbfb705f13.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


2024-09-26 09:56:09.970249: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-26 09:56:10.015525: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 09:56:10.015562: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 09:56:10.015598: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 09:56:10.023075: I tensorflow/core/platform/cpu_feature_g

Processing image: uploads/image_0.png
Successfully saved annotated images as "outputs/image_0_open_vocabulary_detection.jpg" and "outputs/image_0_open_vocabulary_detection_with_mask.jpg"
Successfully saved mask coordinates to "outputs/image_0_mask.txt"
Successfully saved bounding box coordinates to "outputs/image_0_box.txt"
Processing complete. All images saved to outputs.


2024-09-26 09:58:00.741846: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-26 09:58:00.786937: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 09:58:00.786982: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 09:58:00.787012: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 09:58:00.794624: I tensorflow/core/platform/cpu_feature_g

Processing image: uploads/image_0.png
Successfully saved annotated images as "outputs/image_0_open_vocabulary_detection.jpg" and "outputs/image_0_open_vocabulary_detection_with_mask.jpg"
Successfully saved mask coordinates to "outputs/image_0_mask.txt"
Successfully saved bounding box coordinates to "outputs/image_0_box.txt"
Processing image: uploads/image_1.png
Successfully saved annotated images as "outputs/image_1_open_vocabulary_detection.jpg" and "outputs/image_1_open_vocabulary_detection_with_mask.jpg"
Successfully saved mask coordinates to "outputs/image_1_mask.txt"
Successfully saved bounding box coordinates to "outputs/image_1_box.txt"
Processing image: uploads/image_2.png
Successfully saved annotated images as "outputs/image_2_open_vocabulary_detection.jpg" and "outputs/image_2_open_vocabulary_detection_with_mask.jpg"
Successfully saved mask coordinates to "outputs/image_2_mask.txt"
Successfully saved bounding box coordinates to "outputs/image_2_box.txt"
Processing complete. A